# A. State preparation with Jaynes-Cummings controls

In [1]:
# ruff: noqa
import os

os.sys.path.append("..")
from feedback_grape.fgrape import optimize_pulse_with_feedback
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

As a preliminary step, we consider state preparation
of a target state starting from a pure state. In addition,
we assume that any coupling to an external environment
is negligible and that the parametrized controls can be
implemented perfectly.

Here no feedback is required, we are just testing the parameterized gates setup.

As a first example, we consider the state preparation
of a cavity resonantly coupled to an externally driven
qubit

Here, we consider a particular sequence of
parametrized unitary gates originally introduced by Law
and Eberly

In [2]:
N_cav = 10

In [3]:
def qubit_unitary(alpha_re, alpha_im):
    alpha = alpha_re + 1j * alpha_im
    return tensor(
        identity(N_cav),
        expm(-1j * (alpha * sigmap() + alpha.conjugate() * sigmam()) / 2),
    )

In [4]:
def qubit_cavity_unitary(beta_re, beta_im):
    beta = beta_re + 1j * beta_im
    return expm(
        -1j
        * (
            beta * (tensor(destroy(N_cav), sigmap()))
            + beta.conjugate() * (tensor(create(N_cav), sigmam()))
        )
        / 2
    )

In [5]:
# Uq = qubit_unitary(0.1, 0.1)
# Uqc = qubit_cavity_unitary(0.2, 0.2)
# print(
#     "Uq unitary check:",
#     jnp.allclose(Uq.conj().T @ Uq, jnp.eye(Uq.shape[0]), atol=1e-7),
# )
# print(
#     "Uqc unitary check:",
#     jnp.allclose(Uqc.conj().T @ Uqc, jnp.eye(Uqc.shape[0]), atol=1e-7),
# )

In [6]:
# qubit_unitary(0.1, 0.1).shape

In [7]:
# qubit_unitary(0.1, 0.1)

In their groundbreaking work, Law and Eberly have
shown that any arbitrary superposition of Fock states with
maximal excitation number N can be prepared out of the
ground state in a sequence of N such interleaved gates,
also providing an algorithm to find the correct angles and
interaction durations

In [8]:
# print(qubit_cavity_unitary(0.1, 0.1).shape)

In [9]:
# qubit_cavity_unitary(0.1, 0.1)

## First target is the state $ | 1 ⟩ + | 3 ⟩ $ 

In [10]:
# TODO: Why if time steps are less than 5 then it plateus at 0.5
time_steps = 5  # corressponds to maximal excitation number of an arbitrary Fock State Superposition

In [11]:
psi0 = tensor(basis(N_cav), basis(2))
psi0 = psi0 / jnp.linalg.norm(psi0)
psi_target = tensor((fock(N_cav, 1) + fock(N_cav, 3)) / jnp.sqrt(2), basis(2))
psi_target = psi_target / jnp.linalg.norm(psi_target)

In [12]:
psi0.shape

(20, 1)

In [13]:
psi_target.shape

(20, 1)

In [14]:
print(fock(N_cav, 1))

[[0.+0.j]
 [1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]


Law and Eberly provided an algorithm to determine the correct parameters for state preparation. These include:

- The rotation angle $ |\alpha| $,
- The azimuthal angle $ \arg\left(\frac{\alpha}{|\alpha|}\right) $,
- The interaction duration $ |\beta| $. <br>

So Goal is to find the best control vector (rather than control amplitudes, this time) that leads to finding the optimal state-preparation strategies. Performing as well as the Law-Eberly algorithm.

In [15]:
# TODO: see why if initial [paramaters are complex, adam fidelity is so bad and l-bfgs does error]
import jax.random as random

key = random.PRNGKey(0)  # Random seed for reproducibility
num_gates = len(
    [qubit_unitary, qubit_cavity_unitary]
)  # Number of parameterized gates
# doing a .tolist() will lead to weird shape if you multiply outside with 5 or sth
# playing wiht the range will increase or decrease fidelity
initial_parameters = random.uniform(
    key,
    shape=(time_steps, num_gates, 2),
    minval=-2 * jnp.pi,
    maxval=2 * jnp.pi,
)

print(f"Initial parameters: \n {initial_parameters}")

Initial parameters: 
 [[[-1.02469816 -3.56513646]
  [ 5.84740194  0.93620132]]

 [[ 0.40496999 -1.82331528]
  [ 4.81303274  1.66580299]]

 [[ 0.42474429 -3.8747916 ]
  [ 4.45071507 -5.53624915]]

 [[ 1.8019134   6.20868141]
  [ 1.28195345 -1.40346063]]

 [[-2.55463728  1.8038256 ]
  [-0.97616704  5.45027624]]]


## Optimizing
Currently l-bfgs with the same learning rate of 0.3 converges at a local minimum of 0.5, adam also converges at 0.5 but at smaller learning rates

In [46]:
from feedback_grape.utils.fidelity import ket2dm

result = optimize_pulse_with_feedback(
    U_0=ket2dm(psi0),
    C_target=ket2dm(psi_target),
    parameterized_gates=[qubit_unitary, qubit_cavity_unitary],
    initial_params=initial_parameters,
    num_time_steps=time_steps,
    max_iter=1000,
    convergence_threshold=1e-16,
    type="density",
    mode="no-feedback",
    decay={
        "decay_indices": [0],  # indices of gates before which decay occurs
        # c_ops need to be tensored with the identity operator for the cavity
        # because it is used directly in the lindblad equation
        "c_ops": {
            # weird behavior when gamma is 0.00
            "tm": [tensor(identity(N_cav), jnp.sqrt(0.0015) * sigmam())],
            # "tc": [tensor(identity(N_cav), jnp.sqrt(0.15) * sigmap())],
        },  # c_ops for each decay index
        "tsave": jnp.linspace(0, 1, 2),  # time grid for decay
        "Hamiltonian": None,
    },
    goal="fidelity",
    learning_rate=0.1,
    batch_size=1,
    eval_batch_size=1
)

Iteration 0, Loss: -0.442678
Iteration 10, Loss: -0.676171
Iteration 20, Loss: -0.699814
Iteration 30, Loss: -0.704269
Iteration 40, Loss: -0.706648
Iteration 50, Loss: -0.707260
Iteration 60, Loss: -0.710230
Iteration 70, Loss: -0.730405
Iteration 80, Loss: -0.892387
Iteration 90, Loss: -0.931633
Iteration 100, Loss: -0.979205
Iteration 110, Loss: -0.987293
Iteration 120, Loss: -0.992573
Iteration 130, Loss: -0.995415
Iteration 140, Loss: -0.997014
Iteration 150, Loss: -0.997986
Iteration 160, Loss: -0.998040
Iteration 170, Loss: -0.998088
Iteration 180, Loss: -0.998128
Iteration 190, Loss: -0.998135
Iteration 200, Loss: -0.998136
Iteration 210, Loss: -0.998137
Iteration 220, Loss: -0.998138
Iteration 230, Loss: -0.998138
Iteration 240, Loss: -0.998139
Iteration 250, Loss: -0.998139
Iteration 260, Loss: -0.998139
Iteration 270, Loss: -0.998139
Iteration 280, Loss: -0.998140
Iteration 290, Loss: -0.998140
Iteration 300, Loss: -0.998140
Iteration 310, Loss: -0.998141
Iteration 320, Loss

In [41]:
jnp.array(result.returned_params)

Array([[[[-3.38945462, -5.29055896]],

        [[ 7.59727681,  3.95166575]]],


       [[[ 0.96442159, -2.98989872]],

        [[ 6.76766776,  0.79128691]]],


       [[[ 2.65251138, -1.68338586]],

        [[ 4.04746537, -5.99906467]]],


       [[[ 2.5490795 ,  5.74287501]],

        [[ 1.9558422 , -1.72478531]]],


       [[[-3.08320719, -0.60285763]],

        [[ 1.37681839,  2.82382285]]]], dtype=float64)

In [42]:
print(result.final_state)

[[[-3.31007827e-09-4.13880272e-09j]
  [-9.69749680e-09-2.91289452e-08j]
  [-4.91438200e-02+7.05396972e-01j]
  [ 1.90257013e-08+1.48748850e-08j]
  [ 1.10956854e-08+2.38929704e-09j]
  [ 2.15830409e-10-4.11849521e-09j]
  [-4.91438021e-02+7.05396969e-01j]
  [ 3.56953959e-08+2.38083755e-10j]
  [-4.47670598e-10-3.23544002e-10j]
  [-1.41555124e-09+2.45399712e-09j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 1.01319253e-10+1.89350464e-10j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]]]


In [38]:
print(result)

FgResult(optimized_trainable_parameters=Array([[[ 0.35574144, -3.1213862 ],
        [ 5.52320276,  0.04637444]],

       [[ 1.17155998, -2.91497024],
        [ 3.02387257, -0.85193806]],

       [[-0.36837819, -3.11992014],
        [ 5.70554269, -6.81201994]],

       [[ 1.8019134 ,  6.20868141],
        [ 1.28195345, -1.40346063]],

       [[-2.55463728,  1.8038256 ],
        [-0.97616704,  5.45027624]]], dtype=float64), iterations=304, final_state=Array([[[-1.64070776e-08+7.46268677e-09j],
        [-4.55777746e-08-1.85329552e-08j],
        [ 4.13016667e-01-9.10723467e-01j],
        [-9.89489424e-09+3.27659732e-08j],
        [-1.99378311e-09-1.17046183e-09j],
        [ 2.29298705e-08+8.23805923e-08j],
        [ 0.00000000e+00+0.00000000e+00j],
        [-1.42476748e-08+2.39273537e-09j],
        [ 0.00000000e+00+0.00000000e+00j],
        [ 0.00000000e+00+0.00000000e+00j],
        [ 0.00000000e+00+0.00000000e+00j],
        [ 0.00000000e+00+0.00000000e+00j],
        [ 0.00000000e+00+0.000

In [39]:
result.final_fidelity

Array(0.5, dtype=float64)

In [34]:
# from feedback_grape.grape_paramaterized import calculate_trajectory
# from feedback_grape.utils.fidelity import fidelity

# U_final = calculate_trajectory(
#     U_0=psi0,
#     parameters=result.returned_params[0],
#     time_steps=time_steps,
#     parameterized_gates=[qubit_unitary, qubit_cavity_unitary],
#     propcomp="memory-efficient",
#     type="state",
#     decay=None,
# )
# print(
#     fidelity(C_target=psi_target, U_final=result.final_operator, type="state")
# )